In [9]:
# Importing the libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#% matplotlib inline
from datetime import datetime

import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.figure_factory as ff
import plotly.express as px
from plotly import tools
from plotly.subplots import make_subplots
from plotly.offline import iplot

from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
nltk.download('maxent_ne_chunker')
nltk.download('words')

import string
import gensim
import collections
from gensim.corpora import Dictionary

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

from autocorrect import Speller

from bs4 import BeautifulSoup
import requests
import csv
import time

import re
import string
import gc
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/aditidutta/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [12]:
# gathering up all the data from the files received in via Google Scholar search 

df1 = pd.read_csv('data/2012.csv')
df2 = pd.read_csv('data/2013.csv')
df3 = pd.read_csv('data/2014.csv')
df4 = pd.read_csv('data/2015.csv')
df5 = pd.read_csv('data/2016.csv')
df6 = pd.read_csv('data/2017.csv')
df7 = pd.read_csv('data/2018.csv')
df8 = pd.read_csv('data/2019.csv')
df9 = pd.read_csv('data/2020.csv')
df10 = pd.read_csv('data/2021.csv')
df11 = pd.read_csv('data/2022.csv')


df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11], ignore_index=True)
df.head()

,Cites,Authors,Title,Year,Source,Publisher,ArticleURL,CitesURL,GSRank,QueryDate,...,StartPage,EndPage,ECC,CitesPerYear,CitesPerAuthor,AuthorCount,Age,Abstract,FullTextURL,RelatedURL
0,1,M Turno,'Dai che mi vesto da puttana!': Cultural Repre...,2012.0,NaN,Citeseer,https://citeseerx.ist.psu.edu/viewdoc/download...,https://scholar.google.com/scholar?cites=68233...,93,2022-07-17 10:02:25,...,NaN,NaN,1,0.1,1,1,10.0,"Nadia frantically tries to escape, chased by a...",https://citeseerx.ist.psu.edu/viewdoc/download...,https://scholar.google.com/scholar?q=related:x...
1,1,Z Vaisman,A Cognitive-Behavioral Therapy Model for the T...,2012.0,NaN,search.proquest.com,https://search.proquest.com/openview/e9d6eb306...,https://scholar.google.com/scholar?cites=14365...,27,2022-07-17 10:02:25,...,NaN,NaN,1,0.1,1,1,10.0,… to impose such a classification in our next ...,NaN,https://scholar.google.com/scholar?q=related:P...
2,0,J Lammi,A genre analysis of Iain M. Banks' The Player ...,2012.0,NaN,erepo.uef.fi,https://erepo.uef.fi/bitstream/handle/12345678...,NaN,82,2022-07-17 10:02:25,...,NaN,NaN,0,0.0,0,1,10.0,… Sentient artificial intelligence has also be...,https://erepo.uef.fi/bitstream/handle/12345678...,https://scholar.google.com/scholar?q=related:n...
3,31,MR Davidson,A nurse's guide to women's mental health,2012.0,NaN,books.google.com,https://books.google.com/books?hl=en&lr=&id=I3...,https://scholar.google.com/scholar?cites=15456...,37,2022-07-17 10:02:25,...,NaN,NaN,31,3.1,31,1,10.0,Named a 2013 Doody's Core Title! 2012 Third Pl...,NaN,https://scholar.google.com/scholar?q=related:8...
4,3,A Wright,A review of best practices in child welfare,2012.0,NaN,books.google.com,https://books.google.com/books?hl=en&lr=&id=hg...,https://scholar.google.com/scholar?cites=13797...,48,2022-07-17 10:02:25,...,NaN,NaN,3,0.3,3,1,10.0,"… the child welfare field, as BP provide the i...",http://phoenixsinclairinquiry.ca/exhibits/exhi...,https://scholar.google.com/scholar?q=related:h...


In [13]:
# View the columns
df.columns

Index(['Cites', 'Authors', 'Title', 'Year', 'Source', 'Publisher',
       'ArticleURL', 'CitesURL', 'GSRank', 'QueryDate', 'Type', 'DOI', 'ISSN',
       'CitationURL', 'Volume', 'Issue', 'StartPage', 'EndPage', 'ECC',
       'CitesPerYear', 'CitesPerAuthor', 'AuthorCount', 'Age', 'Abstract',
       'FullTextURL', 'RelatedURL'],
      dtype='object')

In [14]:
df[['ArticleURL']].iloc[10]  #not in full-text #difficult to scrape web data

ArticleURL    https://link.springer.com/article/10.1007/s105...
Name: 10, dtype: object

In [15]:
df['Source'].value_counts()

# shows a lot of preprints, and it occupies the largest share of the lot

arXiv preprint arXiv …                                   85
api.taylorfrancis.com                                    34
researchgate.net                                         27
Journal of …                                             26
Proceedings of the …                                     24
                                                         ..
CENTRAL EUROPEAN POLITICAL SCIENCE                        1
The Kaleidoscope of Gender: Prisms, Patterns, and …       1
Busted! The Truth about the 50 Most Common Internet …     1
Psychology, Crime & Law                                   1
CHI Conference on Human Factors in …                      1
Name: Source, Length: 1581, dtype: int64

In [16]:
df['Source'].isna().count() 
# The high number of NA could be because of a large number of preprints

3892